# Convolutional Neural Networks

## Introduction

**Learning Objectives**

- What is a Convolutional Neural Network (CNN), and what makes it different from other types of neural networks?
- How does a CNN recognise patterns in images?
- What are the main parts of a CNN, and what do they do?
- What are the kernel and pooling layers, and how do they work?

In this lesson, we will explore **Convolutional Neural Networks (CNNs)**, a class of neural networks renowned for their proficiency in processing visual data.

Before going into the implementation with **PyTorch** we will explore what makes a **CNN** different from standard neural networks.

## The CNN?

KEY TERMS BOX

A CNN is a type of *artificial neural network* designed specifically for processing and analysing *visual data*, such as **images** and **videos**. What sets CNNs apart from other types of neural networks, such as **fully connected neural networks**, is their unique architecture and ability to effectively capture *spatial hierarchies of features* in images.

## How are CNNs different?

### Why are CNNs mainly used for tasks involving images?

CNNs are mainly used for tasks involving images because they are designed to *automatically* and *adaptively* learn **spatial hierarchies of features** from images. CNNs exploit the 2-D structure of images through local connections and shared weights, effectively capturing *local patterns* like *edges*, *textures* and shapes. This specialised architecture makes them exceptionally good at tasks like **image classification**, **object detection**, and more, where understanding spatial hierarchies of features is crucial.

### Structure

A CNN's structure typically consists of *several layers that perform specific operations* on the input data:

- **Input Layer**: Receives the raw input data, usually images represented as a matrix of pixel values.
- **Convolutional Layers**: These layers apply filters (also known as kernels) to the input image to extract features such as edges, textures, and shapes. Each filter is convolved across the input image, producing a feature map that highlights specific patterns.
- **Activation Function**: After each convolution operation, an activation function (commonly **ReLU** (*Rectified Linear Unit*)) is applied element-wise to introduce *non-linearity*, enabling the network to learn complex patterns.
- **Pooling Layers**: These layers *downsample the feature maps* produced by the convolutional layers, *reducing their spatial dimensions* while retaining important features. **Max pooling** and **average pooling** are common techniques used for this purpose.
- **Fully Connected Layers (Dense Layers)**: After the convolutional and pooling layers, the feature maps are *flattened into a vector* and passed through one or more fully connected layers, i.e. what we learnt in the previous lesson. These layers perform *classification* based on the learnt features, *mapping them to the output classes*.
- **Output Layer**: The final layer of the CNN, typically a **softmax layer** for classification tasks, assigns probabilities to each class label, indicating the *likelihood of the input image belonging to each class*.

![Diagram of CNN Architecture](./images/cnn_architecture_diagram.jpeg)

### How does a CNN recognise patterns in images?

A CNN recognises patterns in images through a series of **convolutional** and **pooling** layers, you may have noticed these as the key difference in architecture between the *vanilla neural network* and the CNN. The **convolutional** layers use filters to capture *spatial features* such as edges and textures by applying these filters across the entire image. Pooling layers then reduce the spatial dimensions of these feature maps, emphasising the presence of features while making the representation more compact and invariant to minor variations and translations. Through successive layers, the network learns to recognise more complex patterns by combining lower-level features. Finally, *fully connected layers aggregate these features* to make **predictions**.

### How are CNNs trained to recognise objects in images?

CNNs are trained to recognise objects in images using *supervised learning*, typically with a large dataset of labeled images. During training, the network learns by adjusting its weights and biases to minimise the difference between its predictions and the actual labels. This process is facilitated by a **loss function** and an **optimisation** algorithm, such as **stochastic gradient descent**. Over time, the network learns to extract and combine features that are predictive of specific classes, enabling it to recognise objects in images.

## Worked example

Just like the previous lesson, we will use the MNIST dataset. Utilising its familiarity to help us understand what is happening in the layers unique to a **CNN**.

### First, lets import the necessary packages:

In [ ]:
import torch # the PyTorch module
import torch.nn.functional as F # the functional module

import warnings
warnings.filterwarnings("ignore") # ignore warnings

# Import the MNIST dataset
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import numpy as np

### Next we will download the data in the exact same way as previously:

In [ ]:
# Normalise the data to have mean 0 and std 1 using dataset statistics
transform = transforms.Compose([
                          transforms.ToTensor(),    # convert the image to a tensor
                          transforms.Normalize((0.1307,), (0.3081,)) # normalise the image
                        ])

# Define a training data loader

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST( 'data', 
                    train=True, 
                    download=True,
                    transform = transform
                    ),
    batch_size=64, shuffle=True)

# Define a test data loader
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST( 'data', 
                    train=False, 
                    download=True,
                    transform = transform
                    ),
    batch_size=1000, shuffle=True)

### And plot them to check everything has worked correctly:

In [ ]:
# Our plotting function:
def plot_images(images, labels):
    """
    Display a 4x4 grid of MNIST images with their corresponding labels.
    
    :param images: Batch of image tensors to display. Each image should be
                   either 28x28 or flattened to 784 pixels.
    :type images: torch.Tensor
    :param labels: Corresponding labels for each image (digit 0-9).
    :type labels: torch.Tensor
    
    :returns: None (displays the plot using plt.show())
    :rtype: None
    """
    fig, axes = plt.subplots(4, 4)
    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i].reshape(28, 28), cmap='binary')
        ax.set(xticks=[], yticks=[], xlabel=int(labels[i]))
        # make a white background for the text
        ax.xaxis.label.set_color('white')
        # increase the vertical distance between the subplots:
        fig.subplots_adjust(hspace=0.5)
    plt.show()


# Plot a few of the MNIST images with their labels:
torch.manual_seed(42)
images, labels = next(iter(train_loader))
plot_images(images, labels)

## Kernels / Filters

KEY TERMS BOX

A **kernel** is a small matrix of **trainable weights** (sometimes called a *sliding window*) that is applied to an image. The kernel is moved across the image, and at each position its values are multiplied - element-wise - with the corresponding image pixels, and then summed. This produces a single value in the output. Repeating this process across the image produces a **feature map**. The values in the kernel are learnt during training.

When training a convolutional neural network, the **weights** in the kernels are learnt. The kernels each learn to *detect specific features* in the image. For example, a kernel might specialise in detecting vertical edges in the image. Many such kernels will be learnt during training.

These convolution operations are repeated as the kernel slides across the **entire image** (*blue*). The output of the convolution is a **feature map** (*green*). **Padding** (*white*) can be added *around* the image to control the size of the output feature map. The kernel itself is learnt during training.

![Convolutional Operation, via wiki.org](./images/cnn-kernel.gif)

### Why are kernels so important?

**Feature Detection**
Each filter in a CNN is designed to detect *specific features* in the input data. In the context of image processing, early layers typically detect simple features such as *edges* or *corners*, whilst deeper layers identify more complex patterns or entire objects. This hierarchical feature extraction process is fundamental to tasks like image classification and object detection.

**Spatial Hierarchies**
Through a series of convolutions, *CNN filters capture the spatial hierarchy* present in images, enabling the network to understand the context and relationships between different regions. For instance, recognising a face involves detecting individual features (such as the eyes, nose and mouth) as well as their *relative spatial arrangement* to one another.

**Parameter Sharing**
One of the key advantages of CNNs is **parameter sharing**: filters *reuse the same parameters* across different parts of the input data. In image processing, this dramatically reduces the number of parameters required compared to fully connected networks, leading to more *efficient learning* and *reduced overfitting*. Parameter sharing also means that once a filter learns to recognise a feature in one region of an image, it can recognise the same feature anywhere else in that image.

**Translation Invariance**
Thanks to the sliding window approach of applying filters, CNNs exhibit a degree of **translation invariance**. If a learnt feature moves to a different location in the input, the CNN can *still* detect it. This property is particularly valuable for image and video analysis, where the precise location of objects can vary significantly across different examples.

**Automatic Feature Engineering**
Perhaps most importantly, CNNs perform **automatic feature engineering**. In traditional machine learning approaches, feature extraction requires domain expertise and must be done manually. CNNs, through their learnt filters, automatically discover the most relevant features from the data during training. This reduces the need for manual feature engineering and allows CNNs to be applied to a wide range of tasks without requiring domain-specific preprocessing.

## Exploring the kernel

To explore the concept of a kernel, let us define a simple **edge detection** kernel.:

$$K = \begin{bmatrix} -1 & -1 & -1 \\ -1 & 8 & -1 \\ -1 & -1 & -1 \end{bmatrix}$$

In [ ]:
# Our 3x3 kernel — detects edges in all directions (Laplacian):
kernel = torch.tensor( [[-1, -1, -1], 
                        [-1, 8, -1], 
                        [-1, -1, -1]], dtype=torch.float)

To apply the kernel to an image, we need to implement the **convolution operation**. 

KEY TERMS BOX

The term **convolution operation** refers to the process of applying a kernel to an image by *sliding* it across the image, computing a weighted sum of the overlapping pixel values at each position.

The function given in the code cell below performs a convolution operation, sliding the kernel across the image and computing the dot product at each position to produce a **feature map** that highlights detected patterns.

At each step, the kernel is multiplied element-wise with the corresponding image patch, and the results are summed. This process reveals where specific features, in this case, edges, appear in the image. The *output will be smaller than the input* because the kernel cannot extend beyond the image boundaries.

In [ ]:
# define a function to apply a kernel to an image
def apply_kernel(image, kernel):
    """
    Apply a convolution kernel to an image using the dot product operation.
    
    NOTE: This implementation uses nested Python loops for clarity. It is
    intentionally slow and is here purely to illustrate what a convolution
    does step-by-step. In practice, PyTorch's built-in F.conv2d() performs
    the same operation orders of magnitude faster.
    
    :param image: Input image tensor with shape (height, width). Typically a 2D
                  grayscale image represented as a PyTorch tensor.
    :type image: torch.Tensor
    :param kernel: Convolution kernel (filter) to apply to the image. Must be smaller
                   than the input image. Common sizes are 3x3, 5x5, or 7x7.
    :type kernel: torch.Tensor
    
    :returns: Feature map (convolved output) with reduced dimensions. The output
              height = image_height - kernel_height + 1, and output width =
              image_width - kernel_width + 1.
    :rtype: torch.Tensor
    """
    
    # get the image dimensions
    image_height, image_width = image.shape

    # get the kernel dimensions
    kernel_height, kernel_width = kernel.shape

    # calculate the output dimensions
    output_height = image_height - kernel_height + 1
    output_width = image_width - kernel_width + 1

    # initialise the output tensor
    output = torch.zeros(output_height, output_width)

    # loop through the image and apply the kernel
    for i in range(output_height):
        for j in range(output_width):

            # get the current image patch
            image_patch = image[i:i+kernel_height, j:j+kernel_width]

            # calculate the dot product between the image patch and the kernel
            output[i, j] = torch.dot(image_patch.reshape(-1), kernel.reshape(-1))

    return output

Taking the third image from the batch we loaded previously, `images[2][0]`, we can show what happens when we apply the kernel to it. 

In [ ]:
# make a subplot of three images (original, kernel, output): Make the images bigger
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# plot the original image
axes[0].imshow(images[2][0], cmap='binary')
axes[0].set_title(f'original image ({images[2][0].shape[0]}x{images[2][0].shape[1]})')

# plot the kernel
axes[1].imshow(kernel, cmap='binary')
axes[1].set_title('kernel (3x3)')

# apply the kernel to the image
feature_map = apply_kernel(images[2][0], kernel)

# plot the output
axes[2].imshow(feature_map, cmap='binary')
axes[2].set_title(f'output or `feature map` ({feature_map.shape[0]}x{feature_map.shape[1]})')

plt.show()

The rightmost plot shows the result of applying our **edge detection kernel**. See how the feature map highlights the edges and contours of the digit in remarkable detail. The kernel has successfully identified regions where pixel intensities change rapidly. Only using this simple kernel convolution we can already see the power behind convolutional filters.

NOTE BOX

The *feature map* has been reduced by 2 in each dimension. This is because the filter ($3 \times 3$) cannot center on the edge pixels when sliding across the image. Therefore, a pixel from each edge of the image is lost.

## Illustrating kernels

To further illustrate the above code we will have a closer look into the image by taking a patch from the above input image:

In [ ]:
# Two points in the image
a = 14
b = 8
# get the 3x3 sized patch from the image
patch = images[0][0][a:a+3, b:b+3]

# plot the patch and label each pixel with the corresponding value
fig, ax = plt.subplots(figsize=(5,5))
ax.imshow(patch, cmap='binary_r')
for i in range(patch.shape[0]):
    for j in range(patch.shape[1]):
        ax.text(j, i, round(patch[i, j].item(), 2), ha='center', va='center', color='purple', fontsize=14)
ax.title.set_text('patch')
plt.show()

We are now viewing the pixels as their numerical values - much in the same way as the kernel approaches it.

REMEMBER BOX

We have *normalised* our pixel values when we loaded in the data. This means that the values have a mean of 0. Darker pixels tend to be more negative, and lighter pixels tend to be more positive. In the case of the plot the smallest and largest values set the scale of the colour bar, so it is not a true representation of the true colours in the image.

Using just this patch we can break down the convolution operation into three simple steps:

1. **Select a patch** from the image (a 3×3 region) - **Done**
2. **Multiply** each patch value by the corresponding kernel value (element-wise multiplication)
3. **Sum** all the products to produce a single output value

Our edge detection kernel (with value 8 in the center and -1 around the edges) responds strongly when positioned over regions where pixel intensities change rapidly, such as where edges occur. In uniform regions, the positive and negative values cancel out, producing a weak response.

Below, we can see how the element-wise multiplication represents changes in numerical values.

In [ ]:
# Show the element-wise multiplication between patch and kernel
element_wise_product = patch * kernel

fig, ax = plt.subplots(figsize=(5,5))
ax.imshow(element_wise_product, cmap='RdBu_r')  # Red-Blue colormap to show positive/negative
for row in range(element_wise_product.shape[0]):
    for col in range(element_wise_product.shape[1]):
        ax.text(col, row, round(element_wise_product[row, col].item(), 2), 
                ha='center', va='center', color='black', fontsize=14)
ax.title.set_text('Element-wise multiplication (patch × kernel)')
plt.show()

print(f'Sum of all products: {element_wise_product.sum().item():.3f}')

PRACTICE EXERCISE

Download the notebook and define a **vertical** and **horizontal** edge detection kernel using the matrices below:

$$K_{vertical} = \begin{bmatrix} -1 & 0 & 1 \\ -2 & 0 & 2 \\ -1 & 0 & 1 \end{bmatrix} \qquad K_{horizontal} = \begin{bmatrix} -1 & -2 & -1 \\ 0 & 0 & 0 \\ 1 & 2 & 1 \end{bmatrix}$$

Apply both to the same MNIST image used in the notebook using `apply_kernel`, then plot all three feature maps side by side: Laplacian (the previous version), vertical edges, horizontal edges.

How do the outputs differ from one another? Can you explain why the Laplacian alone captures what the two directional kernels achieve together?

In [ ]:
# Write your code here:


## Pooling

After the convolutional layer, we need to apply a **pooling layer** to reduce the size of the feature map.

Pooling operations are used in convolutional neural networks to reduce the size of the feature maps. This is done to reduce the number of parameters in the network, and to reduce the amount of computation required.

There are two types of pooling operations:
- **max pooling**: This takes the maximum value from a region of the feature map.

- **average pooling**: Average pooling takes the average value from a region of the feature map. 

The size of the region is defined by the **pooling kernel**. The pooling kernel is moved across the feature map, and the max or average value is computed for each region. The *output* of the pooling operation is a **feature map**. The pooling kernel is *not* learnt during training.

KEY TERMS BOX

A **pooling operation** reduces the spatial size of a feature map by summarising values within small regions: for instance, by taking the maximum or average value in each region.

### How do pooling layers help in CNNs?

**Dimensionality Reduction**
Pooling layers reduce the spatial dimensions (height and width) of the input volume for the subsequent convolutional layers. This reduction in size helps decrease the computational load and the number of parameters in the network, leading to *faster training* and *less risk of overfitting*.

**Invariance to Small Translations**
By summarising the presence of features in patches of the input image, pooling (especially max pooling) provides a form of *translation invariance*. This means the network can recognise features regardless of their exact location in the input, making CNNs *robust to small shifts and distortions in the input image*.

**Feature Generalisation**
Pooling *helps the network to generalise better* by abstracting the input features. Instead of learning from the high-resolution data, pooling layers help the network to focus on the presence or absence of features in a more general form. This abstraction is critical for the network to recognise objects and patterns efficiently in various contexts and variations.

**Preserving Dominant Features**
Pooling layers, particularly max pooling layers, preserve the strongest features, filtering out weaker and less informative ones. This selective feature preservation ensures that the network *focuses on the most relevant attributes* of the input data for the prediction task, at hand.

REMEMBER BOX

**Generalisation** is a *crucial* component of machine learning. A lot of the additional components in a neural network model are mostly there to stop it from overfitting to the training data. The models themselves are very powerful in learning specific patterns, and it is our job to make sure they learn the *general* patterns, and *not* the data-specific patterns. That is why the use of *noise*, or *pooling*, are are so effective.

## Illustration pooling

Below we have applied $2 \times 2$ *max pooling* to our edge-detected feature map, reducing its dimensions by half, while keeping the most prominent edges.

In [ ]:
# Shape before pooling
print("Original feature map shape:", feature_map.shape)
print(f"Original feature map dimensions: {feature_map.shape[0]}×{feature_map.shape[1]}\n")

# Apply max pooling with a 2x2 kernel and stride of 2 - reducing the dimensions by half
pooled_output = F.max_pool2d(feature_map.unsqueeze(0).unsqueeze(0), kernel_size=2, stride=2)
pooled_output = pooled_output.squeeze()

# Shape after pooling
print(f"Pooled feature map shape: {pooled_output.shape}")
print(f"Pooled feature map dimensions: {pooled_output.shape[0]}×{pooled_output.shape[1]}\n")

Just from the shape alone, we can see that the size of the data has halved. Visualising this, the feature map becomes less detailed, but still retains the same *general* information. Fine-grained spatial detail is discarded, making the data simpler and easier for the network to process.

In [ ]:
# Visualise the pooling operation
fig, axes = plt.subplots(1, 3, figsize=(12, 6))

# Original feature map
axes[0].imshow(feature_map, cmap='gray')
axes[0].set_title(f'Original Feature Map\n({feature_map.shape[0]}×{feature_map.shape[1]})', fontsize=14)
axes[0].axis('off')

# Pooled feature map
axes[1].imshow(pooled_output, cmap='gray')
axes[1].set_title(f'After Max Pooling (2×2)\n({pooled_output.shape[0]}×{pooled_output.shape[1]})', fontsize=14)
axes[1].axis('off')

# Side-by-side comparison with grid overlay
axes[2].imshow(feature_map, cmap='gray', alpha=0.7)
# Draw grid to show pooling regions
for i in range(0, feature_map.shape[0], 2):
    axes[2].axhline(y=i-0.5, color='red', linewidth=2)
for j in range(0, feature_map.shape[1], 2):
    axes[2].axvline(x=j-0.5, color='red', linewidth=2)
axes[2].set_title('Original with Pooling Regions\n(2×2 blocks)', fontsize=14)
axes[2].axis('off')

plt.tight_layout()
plt.show()

## Summary

In this lesson, we have expanded upon how the vanilla neural network can be modified to become a **convolutional neural network**. This is a *more complex* model that is able to learn features from images and other data *via* the use of **kernel filters** to extract *local patterns* and **pooling** to reduce the size of the feature maps.